In [ ]:
import pandas as pd
#import re
#import nltk
#from nltk.corpus import stopwords

#nltk.download("stopwords")
#stop_words = set(stopwords.words("english"))

# lscs members
lscs = pd.read_csv(
    "lscs_members.csv",  # 👈 disable quoting
    on_bad_lines="warn"      # skip/warn bad rows
)

lscs.head()


: 

In [24]:
# houses
houses = {
    "Red": {
        "House Name": "House of Jasper",
        "Gem/Crystal": "Jasper",
        "Signature Color/Look": "earthy red/brown-red"
    },
    "Orange": {
        "House Name": "House of Carnelian",
        "Gem/Crystal": "Carnelian",
        "Signature Color/Look": "fiery orange"
    },
    "Yellow": {
        "House Name": "House of Citrine",
        "Gem/Crystal": "Citrine",
        "Signature Color/Look": "golden yellow"
    },
    "Green": {
        "House Name": "House of Aventurine",
        "Gem/Crystal": "Aventurine",
        "Signature Color/Look": "shimmering green"
    },
    "Light Blue": {
        "House Name": "House of Amazonite",
        "Gem/Crystal": "Amazonite",
        "Signature Color/Look": "turquoise blue-green"
    },
    "Indigo/Dark Blue": {
        "House Name": "House of Lapis Lazuli",
        "Gem/Crystal": "Lapis Lazuli",
        "Signature Color/Look": "deep royal blue"
    },
    "Purple": {
        "House Name": "House of Amethyst",
        "Gem/Crystal": "Amethyst",
        "Signature Color/Look": "violet to purple"
    },
    "White": {
        "House Name": "House of Moonstone",
        "Gem/Crystal": "Moonstone",
        "Signature Color/Look": "milky white with rainbow sheen"
    },
    "Black": {
        "House Name": "House of Obsidian",
        "Gem/Crystal": "Obsidian / Onyx",
        "Signature Color/Look": "jet black (volcanic glass)"
    },
    
    "Pink/Brown": {
        "House Name": "House of Rose Quartz",
        "Gem/Crystal": "Serpentine",
        "Signature Color/Look": "pink"
    }
}

house_df = pd.DataFrame.from_dict(houses, orient="index")
house_df.index.name = "Color Category"
house_df.to_csv("houses.csv", index=True) 
house_df

,House Name,Gem/Crystal,Signature Color/Look
Color Category,,,
Red,House of Jasper,Jasper,earthy red/brown-red
Orange,House of Carnelian,Carnelian,fiery orange
Yellow,House of Citrine,Citrine,golden yellow
Green,House of Aventurine,Aventurine,shimmering green
Light Blue,House of Amazonite,Amazonite,turquoise blue-green
Indigo/Dark Blue,House of Lapis Lazuli,Lapis Lazuli,deep royal blue
Purple,House of Amethyst,Amethyst,violet to purple
White,House of Moonstone,Moonstone,milky white with rainbow sheen
Black,House of Obsidian,Obsidian / Onyx,jet black (volcanic glass)


In [26]:
specific = lscs[lscs["full_name"].str.contains("Rohann", case=False)]
specific

,id,full_name,nickname,email,telegram,position_id,committee_id,college,program,discord,interests,contact_number,fb_link
254,12343765,"Dizon, Rohann Gabriel D.",NaN,rohann_dizon@dlsu.edu.ph,NaN,VP,RND,College of Computer Studies,BS - CS ST,NaN,Volleyball,0967 087 1244,https://www.facebook.com/roedzn


In [16]:
no_interests = lscs[lscs["interests"].str.contains("nan")].sort_values("full_name")
no_interests

,id,full_name,nickname,email,telegram,position_id,committee_id,college,program,discord,interests,contact_number,fb_link
302,12409286,"Abogadie, Nina Rica T.",NaN,nina_abogadie@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS - Computer Science,NaN,nan,0906 014 8362,https://www.facebook.com/natt.abogadie
142,12306479,"Aguarin, Trish Ann Danielle C.",NaN,trish_ann_aguarin@dlsu.edu.ph,NaN,AVP,HRD,CCS,BSMS CS,NaN,nan,0908 8911 0870,https://www.facebook.com/trishanndanielle.aguarin
49,12206784,"Alabat, Jeanne Hazel W.",NaN,jeanne_alabat@dlsu.edu.ph,NaN,MEM,MEM,College of Computer Studies,BSMS - Computer Science,NaN,nan,09568546712,https://www.facebook.com/jeannehazel.alabat.5
247,12342181,"Albo, Leigh Gwyneth M.",NaN,leigh_albo@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS - Computer Science (Challenge-Based Learning),NaN,nan,9973761929,facebook.com/leigh.gwynth
441,12413356,"Almeda, Angelo Florence L.",NaN,angelo_florence_almeda@dlsu.edu.ph,NaN,AVP,PUBS,CCS,BS - Information Technology,NaN,nan,0976 596 8795,https://www.facebook.com/share/1BCfaKc9J7/
...,...,...,...,...,...,...,...,...,...,...,...,...,...
513,12415375,"Xu, Kai Wen H.",NaN,kai_xu@dlsu.edu.ph,NaN,MEM,MEM,College of Computer Studies,BS CS - CSE,NaN,nan,0915 530 6388,https://www.facebook.com/kaiwen.xu.5201/
230,12336874,"YU, Bernard Jacob V.",NaN,bernard_yu@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS - Information Technology,NaN,nan,0917 115 7179,https://www.facebook.com/yu.bernard.jacob/
576,12486906,"Yu, Nathan Mitchell, M",NaN,nathan_yu@dlsu.edu.ph,NaN,MEM,MEM,CCS,CS NIS,NaN,nan,09178797444,https://www.facebook.com/share/19XixtQ6tD/
179,12308390,"Zulueta, Isha Daphne E.",NaN,isha_daphne_zulueta@dlsu.edu.ph,NaN,MEM,MEM,College of Computer Science,BS - Computer Science Major in Network and Inf...,NaN,nan,0956 681 0678,https://www.facebook.com/isha.zulueta.3


In [29]:
# Clean your "interests" column
lscs["cleaned_interests"] = (
    lscs["interests"]
    .astype(str)
    .replace(r"[&/;\n|*+-]", ",", regex=True)  # added + and -
    .str.replace(r",+", ",", regex=True)        # collapse multiple commas
    .str.strip(", ")                            # strip leading/trailing commas/spaces
)

lscs['cleaned_interests'].sort_values()

746    1. Coding (I worked on a platform game with tw...
789                        3D Modeling and Video Editing
5                                                      ?
708    ACADEMIC,Math, Science, History, Philosophy,MU...
777    Anime, Deepwoken, Overwatch, Pokémon, Genshin ...
                             ...                        
37                                       what if ayaw ko
56                                               writing
802                 writing, drawing, tech, games, music
456                     writing, graphic design, walking
156                              zenless zone zero, dogs
Name: cleaned_interests, Length: 853, dtype: object